<a href="https://colab.research.google.com/github/sankalpdayal5/federated-learning-on-raspberry-pi/blob/master/helenabarmer/FederatedLearningRaspberryPIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Install PySyft in Google Colab

!pip install tf-encrypted==0.5.6
!pip install msgpack==0.6.1

! URL="https://github.com/openmined/PySyft.git" && FOLDER="PySyft" && if [ ! -d $FOLDER ]; then git clone -b dev --single-branch $URL; else (cd $FOLDER && git pull $URL && cd ..); fi;

!cd PySyft; python setup.py install  > /dev/null

import os
import sys
module_path = os.path.abspath(os.path.join('./PySyft'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
!pip install --upgrade --force-reinstall lz4
!pip install --upgrade --force-reinstall websocket
!pip install --upgrade --force-reinstall websockets
!pip install --upgrade --force-reinstall zstd

     |████████████████████████████████| 1.4MB 4.6MB/s 
     |████████████████████████████████| 266kB 44.8MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44105 sha256=c7b6b43ee194d5cc40415e93e1ab3b21c78dd7e749395d20dc607cd8e500494d
  Stored in directory: /root/.cache/pip/wheels/d9/45/dd/65f0b38450c47cf7e5312883deb97d065e030c5cca0a365030
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 256kB 4.8MB/s 
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
      Successfully uninstalled msgpack-0.5.6
Cloning into 'PySyft'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 30203 (delta 0), reused 4 (delta 0), pack-reused 30190
Receiving objects: 100% (30203/30203), 32.93 MiB | 20.07 MiB/s, done.
Reso

In [0]:
from __future__ import unicode_literals, print_function, division
from torch.utils.data import Dataset

import torch
from io import open
import glob
import os
import numpy as np
import unicodedata
import string
import random
import torch.nn as nn
import time
import math
import syft as sy
import pandas as pd
import random
from syft.frameworks.torch.federated import utils

from syft.workers import WebsocketClientWorker
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

W0728 19:00:26.781474 139671392446336 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0728 19:00:26.806290 139671392446336 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
!wget https://download.pytorch.org/tutorial/data.zip  

--2019-07-28 19:00:30--  https://download.pytorch.org/tutorial/data.zip
Resolving download.pytorch.org (download.pytorch.org)... 13.226.42.62, 13.226.42.10, 13.226.42.64, ...
Connecting to download.pytorch.org (download.pytorch.org)|13.226.42.62|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2882130 (2.7M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   2.75M  15.6MB/s    in 0.2s    

2019-07-28 19:00:30 (15.6 MB/s) - ‘data.zip’ saved [2882130/2882130]



In [1]:
!unzip data.zip

unzip:  cannot find or open data.zip, data.zip.zip or data.zip.ZIP.


In [2]:
path = '/content/data/names/*.txt'

all_letters = string.ascii_letters + ".,;'"
n_letters = len(all_letters)


#Load files in the path
def findFiles(path):
  return glob.glob(path)

#Read a file and then split to lines
def readLines(filename):
  lines =open(filename, encoding='utf-8').read().strip().split('\n')
  return [unicodeToAscii(line) for line in lines]

#Convert  string to ASCII format
def unicodeToAscii(s):
  return ''.join(
      c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn'
      and c in all_letters
  )



category_lines = {}
all_categories = []

for filename in findFiles(path):
  #print(filename)
  category = os.path.splitext(os.path.basename(filename))[0]
  all_categories.append(category)
  lines = readLines(filename)
  category_lines[category] = lines
  
n_categories = len(all_categories)

#for names in glob.glob(path):
  #print(names)
  
 
print("Number of categories: " + "\n" + str(n_categories))
print("\n" + "All categories: ")
print(*all_categories, sep = "\n")

NameError: ignored

In [0]:
print(*category_lines['Polish'][:6], sep = "\n")

In [0]:
class LanguageDataset(Dataset):
  # Constructor
  def __init__(self, text, labels, transform=None):
    self.data = text
    self.targets = labels # categories
    #self.to_torchtensor()
    self.transform = transform
    
  def to_torchtensor(self):
    self.data = torch.from_numpy(self.text, requires_grad=True)
    self.labels = torch.from_numpy(self.targets, requires_grad=True)
  
  # Returns length of dataset/batches
  def __len__(self):
    return len(self.data)
  
  # Returns data and target[torch tensor ]
  def __getitem__(self, idx):
    sample = self.data[idx]
    target = self.targets[idx]
    
    if self.transform:
      sample = self.transform(sample)
      
      return sample, target
    
  

In [0]:
# Arguments for the program
class Arguments():
  def __init__(self):
    self.batch_size = 1
    self.learning_rate = 0.005
    self.epochs = 10000
    self.federate_after_n_batches =15000
    self.seed = 1
    self.print_every = 200
    self.plot_every = 100
    self.use_cuda = False
    
args = Arguments()
    

In [0]:
%%latex

\begin{split}
names\_list = [d_1,...d_n]  \\

category\_list = [c_1,...c_n]
\end{split}

In [0]:
names_list = []
category_list = []

for nation, names in category_lines.items():
  for name in names:
    names_list.append(name)
    category_list.append(nation)
    
print(*names_list[:5], sep = "\n")
print(*category_list[:5], sep = "\n")
print("\n")
print("Data points loaded: " + str(len(names_list)))



In [0]:
# An integer to every category
categories_numerical = pd.factorize(category_list)[0]

# Categories with tensor
category_tensor = torch.tensor(np.array(categories_numerical), dtype=torch.long)

categories_numpy = np.array(category_tensor)

print(names_list[100:120])
print(categories_numpy[100:120])
